# Goal: read in XML data from Perseus treebank

In [1]:
import os
import xml.etree.ElementTree as ET
import pandas as pd

In [2]:
DATA_DIR = '../data/raw/treebank_data-master/v2.0/Greek/nonArethusaCompliant/'
files = os.listdir(DATA_DIR)

In [3]:
files[:2]

['tlg0003.tlg001.perseus-grc1.1.tb.xml', 'tlg0059.tlg001.perseus-grc1.tb.xml']

In [56]:
data = []
for file in files:
    file_dir = os.path.join(DATA_DIR, file)

    print(f"Processing {file}")
    
    # Load XML
    tree = ET.parse(file_dir)
    root = tree.getroot()
    
    # Iterate through sentences
    for sentence in root.iter('sentence'):
        sentence_attrib = sentence.attrib
        sentence_attrib["sentence_id"] = sentence_attrib.pop("id")
        for word in sentence:
            word_attrib = word.attrib
            word_attrib.update(sentence_attrib)
            data.append(word_attrib)

df = pd.DataFrame(data)

Processing tlg0003.tlg001.perseus-grc1.1.tb.xml
Processing tlg0059.tlg001.perseus-grc1.tb.xml
Processing tlg0085.tlg004.perseus-grc2.tb.xml
Processing tlg0540.tlg015.perseus-grc1.tb.xml
Processing tlg0085.tlg007.perseus-grc1.tb.xml
Processing tlg0011.tlg005.perseus-grc2.tb.xml
Processing tlg0096.tlg002.opp-grc2.1-53.tb.xml
Processing tlg0012.tlg002.perseus-grc1.tb.xml
Processing tlg0013.tlg002.perseus-grc1.tb.xml
Processing tlg0020.tlg003.perseus-grc1.tb.xml
Processing tlg0543.tlg001.perseus-grc1.tb.xml
Processing tlg0008.tlg001.perseus-grc1.12.tb.xml
Processing tlg0011.tlg001.perseus-grc2.tb.xml
Processing tlg0085.tlg001.perseus-grc2.tb.xml
Processing tlg0007.tlg004.perseus-grc1.tb.xml
Processing tlg0011.tlg002.perseus-grc2.tb.xml
Processing tlg0085.tlg002.perseus-grc2.tb.xml
Processing tlg0085.tlg003.perseus-grc2.tb.xml
Processing tlg0548.tlg001.perseus-grc1.1.1.1-1.4.1.tb.xml
Processing tlg0020.tlg001.perseus-grc1.tb.xml
Processing tlg0540.tlg014.perseus-grc1.tb.xml
Processing tlg00

In [57]:
df

,id,form,lemma,postag,relation,head,document_id,subdoc,sentence_id,insertion_id,artificial,cite,ref,sg,gloss
0,1,Θουκυδίδης,Θουκυδίδης,n-s---mn-,SBJ,3,urn:cts:greekLit:tlg0003.tlg001.perseus-grc1,1.1.1,1,NaN,NaN,NaN,NaN,NaN,NaN
1,2,Ἀθηναῖος,Ἀθηναῖος,n-s---mn-,ATR,1,urn:cts:greekLit:tlg0003.tlg001.perseus-grc1,1.1.1,1,NaN,NaN,NaN,NaN,NaN,NaN
2,3,ξυνέγραψε,συγγράφω,v3saia---,PRED,0,urn:cts:greekLit:tlg0003.tlg001.perseus-grc1,1.1.1,1,NaN,NaN,NaN,NaN,NaN,NaN
3,4,τὸν,ὁ,l-s---ma-,ATR,5,urn:cts:greekLit:tlg0003.tlg001.perseus-grc1,1.1.1,1,NaN,NaN,NaN,NaN,NaN,NaN
4,5,πόλεμον,πόλεμος,n-s---ma-,OBJ_AP,10,urn:cts:greekLit:tlg0003.tlg001.perseus-grc1,1.1.1,1,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
615667,66,διὰ,διά,r--------,AuxP,69,urn:cts:greekLit:tlg0007.tlg015.perseus-grc1,39.5,431,NaN,NaN,NaN,NaN,NaN,NaN
615668,67,τοῦ,ὁ,l-s---ng-,ATR,68,urn:cts:greekLit:tlg0007.tlg015.perseus-grc1,39.5,431,NaN,NaN,NaN,NaN,NaN,NaN
615669,68,πυρὸς,πῦρ,n-s---ng-,ADV,66,urn:cts:greekLit:tlg0007.tlg015.perseus-grc1,39.5,431,NaN,NaN,NaN,NaN,NaN,NaN
615670,69,ἐξαλλόμενον,ἐξάλλομαι,v-sppema-,ATR,61,urn:cts:greekLit:tlg0007.tlg015.perseus-grc1,39.5,431,NaN,NaN,NaN,NaN,NaN,NaN
